# Searching for faster matrix multiplication algorithms with quantum annealing

In [1]:
import os
import sys
import dimod
import pennylane as qml
from pennylane import qaoa
from pennylane import numpy as np
from matplotlib import pyplot as plt

this_folder = os.path.dirname(os.path.abspath(os.getcwd()))
sys.path.insert(0, this_folder) 

from utils import get_standard_tensor
from ocean_pennylane_integration import *

In [ ]:
dim = 2
initial_tensor = utils.get_standard_tensor(dim) % 2
origo = np.tensordot([0]*4, np.tensordot([0]*4, [0]*4, axes=0), axes=0)

In [ ]:
strassen_tensors = [np.tensordot([0,0,0,1], np.tensordot([-1,0,1,0], [1,0,1,0], axes=0), axes=0),
          np.tensordot([1,1,0,0], np.tensordot([0,0,0,1], [-1,1,0,0], axes=0), axes=0),
           np.tensordot([-1,0,1,0], np.tensordot([1,1,0,0], [0,0,0,1], axes=0), axes=0),
           np.tensordot([1,0,0,1], np.tensordot([1,0,0,1], [1,0,0,1], axes=0), axes=0),
          np.tensordot([0,1,0,-1], np.tensordot([0,0,1,1], [1,0,0,0], axes=0), axes=0),
           np.tensordot([1,0,0,0], np.tensordot([0,1,0,-1], [0,1,0,1], axes=0), axes=0),
           np.tensordot([0,0,1,1], np.tensordot([1,0,0,0], [0,0,1,-1], axes=0), axes=0)]

In [ ]:
tensor = (initial_tensor - strassen_tensors[0] - strassen_tensors[1] - strassen_tensors[2] - strassen_tensors[3]) % 2

# Moving towards origo
while(True):
    bqm = towards_user_defined_small(tensor, origo, dim)
    params, qaoa_circuit, wires = construct_qaoa_and_optimize(bqm, "default.qubit", "SPSA", 1)
    print_probs(qaoa_circuit, wires, params)
    #sample, energy, sampleset = solve_bqm_in_leap(bqm, "Greedy")
    #print(sampleset)
    #x1, y1, z1 = process_result(sample, 2)
    print(x1, y1, z1)
    tensor = (tensor - np.tensordot(x1, np.tensordot(y1, z1, axes=0), axes=0)) % 2
    if np.count_nonzero(tensor.flatten()) == 0:
        print("End")
        break
        
tensor = (initial_tensor + strassen_tensors[0] + strassen_tensors[1] + strassen_tensors[2]) % 2
# Moving towards standard matrix multiplication i.e. the naive method
while(True):
    bqm = towards_user_defined_small(tensor, initial_tensor, dim)
    #sample, energy, sampleset = solve_bqm_in_leap(bqm, "Greedy")
    #print(sampleset)
    #x1, y1, z1 = process_result(sample, 2)
    #print(x1, y1, z1)
    tensor = (tensor - np.tensordot(x1, np.tensordot(y1, z1, axes=0), axes=0)) % 2
    if np.array_equal(tensor, initial_tensor):
        print("End")
        break